# VacationPy-- Using Data from WeatherPy API (Original)
---

##  Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup

import hvplot.pandas
import holoviews as hv
import pandas as pd
import requests

# Import API key
from api_key import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tabou,4.4230,-7.3528,298.59,88,76,0.70,CI,1700718129
1,1,nadi,-17.8000,177.4167,303.15,58,20,5.66,FJ,1700718129
2,2,touros,-5.1989,-35.4608,299.64,69,51,7.08,BR,1700718129
3,3,adamstown,-25.0660,-130.1015,294.44,69,0,4.72,PN,1700717928
4,4,qaqortoq,60.7167,-46.0333,265.57,66,66,2.45,GL,1700718129


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
%%capture --no-display
import hvplot.pandas


# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng', 'Lat',
    geo=True,
    tiles=gvts.OSM,
    hover_cols=['Humidity'],
    color='City',  # Use the 'City' column for coloring
    cmap='Category20',  # Specify a colormap for different colors
    width=1000,
    height=600
)

# Display the map plot
map_plot




:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit the criteria
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] > 300) &
    (city_data_df['Max Temp'] < 500) &
    (city_data_df['Wind Speed'] < 4.5) &
    (city_data_df['Cloudiness'] == 0)
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
58,58,margaret river,-33.9500,115.0667,301.02,44,0,3.44,AU,1700718134
326,326,maxixe,-23.8597,35.3472,302.05,65,0,2.06,MZ,1700718159
371,371,konarka,19.9000,86.1167,301.37,50,0,3.57,IN,1700718163
387,387,chalisgaon,20.4667,75.0167,301.84,36,0,0.66,IN,1700718165
427,427,khawr fakkan,25.3313,56.3420,300.18,47,0,0.44,AE,1700718168
547,547,thandla,23.0000,74.5667,300.95,32,0,2.53,IN,1700718179


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create a DataFrame called hotel_df
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
58,margaret river,AU,-33.9500,115.0667,44,
326,maxixe,MZ,-23.8597,35.3472,65,
371,konarka,IN,19.9000,86.1167,50,
387,chalisgaon,IN,20.4667,75.0167,36,
427,khawr fakkan,AE,25.3313,56.3420,47,
547,thandla,IN,23.0000,74.5667,32,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
import requests
import time
import numpy as np

# Set parameters to search for a hotel
radius = 10000  # You can adjust this value as needed
api_key = geoapify_key 
params = {
    "categories": "accommodation.hotel",
    "filter": "place:5120a11227befefdbf599d89f4d38a3e4a40f00101f9014a7a020000000000c0020692030a4269726d696e6768616d",
    "limit": 1,  # Set the limit to 1 to get only the first hotel result
    "api_key": geoapify_key,  # Replace with your actual Geoapify API key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Check if the response status code is 200 (OK)
    if response.status_code == 200:
        # Convert the API response to JSON format
        name_address = response.json()

        # Grab the first hotel from the results and store the name in the hotel_df DataFrame
        try:
            hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
        except (KeyError, IndexError):
            # If no hotel is found, set the hotel name as "No hotel found".
            hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
        # Log the search results
        print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    else:
        # If the response status code is not 200, print an error message and exit the loop
        print(f"Error: {response.status_code}")
        print(response.content.decode('utf-8'))  # Print response content
        break

    time.sleep(1)  # Add a 1-second delay between requests

# Display sample data
hotel_df


Starting hotel search
margaret river - nearest hotel: Margaret River Hotel
maxixe - nearest hotel: Hotel Herry
konarka - nearest hotel: Hotel Sun Village
chalisgaon - nearest hotel: No hotel found
khawr fakkan - nearest hotel: Raynor Hotel
thandla - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
58,margaret river,AU,-33.9500,115.0667,44,Margaret River Hotel
326,maxixe,MZ,-23.8597,35.3472,65,Hotel Herry
371,konarka,IN,19.9000,86.1167,50,Hotel Sun Village
387,chalisgaon,IN,20.4667,75.0167,36,No hotel found
427,khawr fakkan,AE,25.3313,56.3420,47,Raynor Hotel
547,thandla,IN,23.0000,74.5667,32,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
%%capture --no-display

from geoviews import tile_sources as gvts

# Assuming 'Hotel Name' and 'Country' columns are present in city_data_df
hover_cols = ['Humidity', 'City', 'Hotel Name', 'Country']

# Configure the map plot with OSM tile source
map_plot = city_data_df.hvplot.points(
    'Lng', 'Lat',
    geo=True,
    tiles=gvts.OSM,
    hover_cols=hover_cols,
    color='City',  # Use the 'City' column for coloring
    cmap='Category20',  # Specify a colormap for different colors
    width=1000,
    height=600
)

# Display the map plot
map_plot



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)